# Using thermodynamic analysis to guide metabolic engineering
DO 12-6-2018  
Using Python 3 and eQuilibrator API  


In [2]:
#Changing directory to the github home folder /Ctherm_thermo
import os
os.chdir('/Users/satyakam/Dropbox/work/equilibrator-api-master')
import sys
sys.path.append('/Users/satyakam/Dropbox/work/sbtab-0.9.64')
#sys.path.append('/Users/satyakam/Dropbox/work/equilibrator-api-master')
#sys.path.append('/Users/satyakam/Dropbox/work/equilibrator-api-master/src/equilibrator_api')
sys.path.append('/Users/satyakam/Dropbox/work/equilibrator-api-master/src')
#sys.path.append('/Users/satyakam/Dropbox/work')


In [3]:
import numpy as np
from numpy import array, eye, log, zeros, matrix
from numpy.linalg import inv, solve
import pandas as pd
from equilibrator_api import Reaction, ComponentContribution, ReactionMatcher, CompoundMatcher, ParseError, Pathway
from equilibrator_api.bounds import Bounds
%matplotlib inline

## Set up translator for KEGG IDs
Note: I set these up as dataframes because I was troubleshooting an issue with duplicate KEGG IDs. Eventually I think these should be set up as dictionaries, to make the code more readable

In [4]:
# make a dictionary to translate KEGG IDs into human-readable abbreviations
keggTranslatorDf = pd.read_excel('KEGG_SEED_DO.xls')
kt = keggTranslatorDf #short name for easier typing

# translate KEGG ID to abbreviation
kta = kt.loc[:,['KEGG ID(S)', 'ABBREVIATION']]
kta['KEGG ID(S)'] = kta['KEGG ID(S)'].str.lower() # set to lower case for better matching
kta['ABBREVIATION'] = kta['ABBREVIATION'].str.lower() # set to lower case for better matching
kta.set_index('KEGG ID(S)', inplace = True)

# translate abbreviation to KEGG ID
atk = kt.loc[:,['ABBREVIATION', 'KEGG ID(S)']].sort_values(by = ['KEGG ID(S)'], ascending = True)
atk['KEGG ID(S)'] = atk['KEGG ID(S)'].str.lower() # set to lower case for better matching
atk['ABBREVIATION'] = atk['ABBREVIATION'].str.lower() # set to lower case for better matching
atk = atk.groupby('ABBREVIATION').first() # take the first KEGG ID in each group

atkDict = dict(zip(atk.index, atk['KEGG ID(S)'].str.upper()))
ktaDict = dict(zip(kta.index.str.upper(), kta['ABBREVIATION'].values))

## Set up model
* Choose reactions
* Set fluxes
* Set concentration bounds
* Set pH and ionic strength

In [5]:
os.chdir('/Users/satyakam/Dropbox/work/component_contribution_ctherm')

In [6]:
allRxnDf = pd.read_excel('cth_thermo_model_DO_SD9.xlsx', sheet_name = 'reactions')
allRxnDf

,Name,PlaintextFormula,AbbreviationFormula,KeggFormula,Notes,ppi-pfk,atp-pfk,mal-snt,pdc,gapn,pyk,aldh-ndp,adh-ndp,mal-gapn,tsac
0,ATPase1,ATP + H2O <=> ADP + Pi,h2o + atp <=> adp + pi,C00002 + C00001 <=> C00008 + C00009,ATP hydrolysis for cellobiose transport,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,CBP,Phosphate + Cellobiose <=> D-Glucose + Glucose...,pi + cellb <=> glc-D + g1p,C00009 + C00185 <=> C00031 + C00103,NaN,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,BGL,Cellobiose + H2O <=> 2 D-Glucose,cellb + h2o <=>2 glc-D,C00185 + C00001 <=> 2 C00031,Beta glucosidase,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,GLK-GTP,D-Glucose + GTP <=> D-Glucose-6-phosphate + GDP,glc-D + gtp <=> g6p + gdp,C00031 + C00044 <=> C00092 + C00035,NaN,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,GLK-ATP,D-Glucose + ATP <=> D-Glucose-6-phosphate + ADP,glc-D + atp <=> g6p + adp,C00031 + C00002 <=> C00092 + C00008,NaN,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
5,PGMT,Glucose-1-phosphate <=> D-glucose-6-phosphate,g1p <=> g6p,C00103 <=> C00092,NaN,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
6,PGI,D-Glucose-6-phosphate <=> D-Fructose-6-phosphate,g6p <=> f6p,C00092 <=> C00085,NaN,2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
7,PFK-PPi,PPi + D-fructose-6-phosphate <=> Phosphate + D...,ppi + f6p <=> pi + fdp + h,C00013 + C00085 <=> C00009 + C00354 + C00080,NaN,2,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0
8,PFK-ATP,ATP + D-Fructose-6-phosphate <=> ADP + D-Fruct...,atp + f6p <=> adp + fdp,C00002 + C00085 <=> C00008 + C00354,NaN,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
9,FBA,"D-Fructose-1,6-bisphosphate <=> Glycerone-phos...",fdp <=> dhap + g3p,C00354 <=> C00111 + C00118,NaN,2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [7]:
# choose a flux set, and drop all of the zero-flux reactions
# loop over fluxsets to generate multiple models 
selectedRxnDf={}
fluxSet = 'ppi-pfk'
for f in range(len(fluxSet)):
    selectedRxnDf = allRxnDf.loc[allRxnDf[fluxSet] != 0, ['Name', 'AbbreviationFormula', 'KeggFormula', fluxSet]]
    selectedRxnDf.rename(columns = {fluxSet:'flux'}, inplace = True) # rename the flux columns to 'flux' to simplify subsequent
    #rxnlist[f]= selectedRxnDf  

In [161]:
#Import metabolite data from excel file
Met_bound = pd.read_excel('cth_thermo_model_DO_SD9.xlsx', sheet_name = 'metabolite_bounds')
Alldata = pd.read_excel('./metabolite_data/lt_dataset2_quantified.xlsx')
Met_data = Alldata.loc[Alldata['Replicate'] != 0, ['Timepoint', 'KEGG_ID', 'compound','amount_int','amount','peakAreaTop', 'Sample','Replicate','Maven filename']]
Met_data[:2]

,Timepoint,KEGG_ID,compound,amount_int,amount,peakAreaTop,Sample,Replicate,Maven filename
0,2,C00197,3PG,217.958918,3.400159,8703433.0,A,1,A1_1
1,2,C00197,3PG,137.397616,2.143403,5454995.0,A,2,A1_2


In [9]:
Met_data_WT = pd.read_excel('cth_thermo_model_DO_SD9.xlsx', sheet_name = 'WT_data')
Met_data_WT

,Name,KEGG_ID,Abbreviation,Concentration
0,Glucose-6-phosphate,C00092,g6p,0.008192
1,Fructose-6-phosphate,C00085,f6p,0.001494
2,Glucose-1-phosphate,C00103,g1p,0.006663
3,Fructose 1_6-bisphosphate,C00354,fdp,0.001497
4,Malate,C00149,mal-l,0.037807
5,DHAP,C00111,dhap,0.001270
6,GAP,C00118,g3p,0.000102
7,3-phosphoglycerate,C00197,3pg,0.001348
8,Phosphoenolpyruvate,C00074,pep,0.000690
9,pyruvate,C00022,pyr,0.012655


In [10]:
# parse the KeggFormula column to create eQuilibrator Reaction objects
# create reaction list
reactions = []
for i, row in selectedRxnDf.iterrows():
    rxn = Reaction.parse_formula(row['KeggFormula'], rid = row['Name'])
    if (rxn.check_full_reaction_balancing()):
        reactions.append(rxn)
    else:
        print('Error: reaction {} is not balanced'.format(row['AbbreviationFormula']))
        
# create flux list
fluxes = selectedRxnDf['flux'].values

# need to declare pH and ionic strength constants at the beginning
PH = 7.0
IS = 0.1

# calculate dGO_r_primes
dG0_r_primes = []
for r in reactions:
    result = r.dG0_prime(pH = PH, ionic_strength = IS)
    #print(result)
    dG0_r_primes.append(result)

In [11]:
# set up the model
pp = Pathway(reactions = reactions, fluxes = fluxes, dG0_r_primes = dG0_r_primes)

In [115]:
# Check MDF using Daniel's data
all_data={}
Uf = 0.05 #Uncertainity factor 
DEFAULT_LB = 1e-6
DEFAULT_UB = 2e-2 #20 mM, which is 2x higher than the 10 mM default bound suggested in the Noor et al. 2014 paper.
   
for i, cpd in Met_bound.iterrows():  # reset bounds at the start of each iteration
    pp.bounds.SetBounds(cpd['KEGG_ID'],cpd['Concentration:Min'],cpd['Concentration:Max'])
for i, dat in Met_data_WT.iterrows():
    if dat['KEGG_ID'] not in ['C00002','C00008','C00020','C00035','C00044','C00022','C00103','C00003','C00004','C00005','C00006','C00111','C00118']:
        pp.bounds.SetBounds(dat['KEGG_ID'],dat['Concentration']*(1-Uf),dat['Concentration']*(1+Uf)) 
    #else:
        #pp.bounds.SetBounds(dat['KEGG_ID'],1e-6,1e-2) 
pp.bounds.SetBounds('C00469', 0.47/46, 0.47/46)
pp.bounds.SetBounds('C00033', 8.61/1000, 8.61/1000)       
mdf_data,con = pp.conc_mdf()
print(mdf_data.mdf)
k=0
atp_ratio=1
gtp_ratio=1
nadh_ratio=1
for r in mdf_data.reaction_data:
    print (r.reaction.reaction_id,r.dGr,r.shadow_price)
for c in mdf_data.compound_data:
    print(ktaDict[c.compound], c.concentration,con[k],mdf_data.model.concentration_bounds.GetBoundTuple(c.compound),c.shadow_price)
    k= k+1
    if c.compound in ['C00002']:
        atp_ratio=atp_ratio*c.concentration
    elif c.compound in ['C00008']:
        atp_ratio=atp_ratio/c.concentration
    elif c.compound in ['C00003']:
        nadh_ratio=nadh_ratio/c.concentration
    elif c.compound in ['C00004']:
        nadh_ratio=nadh_ratio*c.concentration
    elif c.compound in ['C00035']:
        gtp_ratio=gtp_ratio/c.concentration
    elif c.compound in ['C00044']:
        gtp_ratio=gtp_ratio*c.concentration
print('atp_ratio =',atp_ratio)
print('gtp_ratio =',gtp_ratio)
print('nadh_ratio =',nadh_ratio)        

0.2082780102208872
ATPase1 -43.50763103236944 0.0
CBP -0.20827801022088632 0.0
GLK-GTP -25.265964422929507 0.0
PGMT -2.237127359538781 3.1164155077197375e-17
PGI -1.6911720565429498 0.0
PFK-PPi -6.0634346769236345 1.3720588385742127e-17
FBA -0.20827801022089432 0.12499999999999996
TPI -0.2082780102208872 0.12499999999999978
GAPDH -0.2082780102208872 0.24999999999999986
PGK-GTP -0.20827801022086945 0.24999999999999972
PGM -0.2082780102208872 0.0
ENO -1.0675753630662963 8.961988617938635e-17
PPDK -7.415631064548181 0.0
PFOR -5.7283289962758435 0.0
RNF_PPiase -36.382776628448056 0.0
ALDH-NADH -0.20827801022085524 0.0
ADH-NADH -1.9993876850997836 0.0
NDK -0.20827801022088943 0.24999999999999972
ATPase2 -49.84663916080911 0.0
PTA -0.2082780102208872 -2.2404971544846587e-17
ACK -7.033219670914847 0.0
h2o 1.0 (1.0, 1.0) (1.0, 1.0) -1.0383646048225674e-16
atp 0.0010000000000000002 (0.0009999999999999994, 0.000999999999999994) (0.001, 0.02) -0.6194066249999991
nad 9.999999999999973e-05 (0.00010

In [57]:
# impose only one concentration bound at a time to check for infeasible bounds
all_data={}
Uf = 0.3 #Uncertainity factor 
DEFAULT_LB = 1e-6
DEFAULT_UB = 2e-2 #20 mM, which is 2x higher than the 10 mM default bound suggested in the Noor et al. 2014 paper.

ref_conc={}
for i, dat in Met_data.iterrows():
    if dat['Timepoint']==5:
        if dat['KEGG_ID'] not in ref_conc:
            ref_conc[dat['KEGG_ID']]={}
            if not ref_conc[dat['KEGG_ID']]:
                ref_conc[dat['KEGG_ID']]=dat['amount_int']
            else:
                ref_conc[dat['KEGG_ID']]=(ref_conc[dat['KEGG_ID']]+dat['amount_int'])/2
for sm in range(12): # divide the samples into datasets
    all_data[sm]={}
    if sm < 6:
        smple = 'A'
    else:
        smple = 'B'
    if (sm)%3==0:
        tym = 2
    elif (sm)%3==1:
        tym = 5
    else:
        tym = 9
    if sm<3 or (sm<9 and sm>5):
        rep = 1
    else:
        rep = 2
    print(smple,tym,rep)
    for i2, dat2 in Met_data_WT.iterrows():
        if dat2['KEGG_ID'] not in ['C00002','C00008','C00020','C00035','C00044','C00022','C00103','C00003','C00004','C00005','C00006']:
            #pp.bounds.SetBounds(dat['KEGG_ID'],dat['Concentration']*.50,dat['Concentration']*1.5)
            if 1:
                for i, cpd in Met_bound.iterrows():  # reset bounds at the start of each iteration
                    pp.bounds.SetBounds(cpd['KEGG_ID'],cpd['Concentration:Min'],cpd['Concentration:Max'] )
                lb={}
                ub={}
                for i, dat in Met_data.iterrows():
                # skip cofactors and impose bound on metsbolites one at a time    
                    if (dat2['KEGG_ID']==dat['KEGG_ID'])and(dat['Sample']==smple and dat['Timepoint']==tym) and (dat['KEGG_ID'] not in ["C00002","C00008","C00020","C00044","C00035","C00003","C00004","C00005","C00006","C00469"]):
                        all_data[sm][i]={}
                        if dat['KEGG_ID'] not in lb:
                            lb[dat['KEGG_ID']]={}
                        if dat['KEGG_ID'] not in ub:
                            ub[dat['KEGG_ID']]={}
                        if not lb[dat['KEGG_ID']]:
                            lb[dat['KEGG_ID']] = dat2['Concentration']*dat['amount_int']/(ref_conc[dat['KEGG_ID']]) *(1-Uf)
                        else:
                            lb[dat['KEGG_ID']] = (lb[dat['KEGG_ID']]+dat2['Concentration']*dat['amount_int']/(ref_conc[dat['KEGG_ID']]) *(1-Uf))/2
                        if not ub[dat['KEGG_ID']]: 
                            ub[dat['KEGG_ID']] = dat2['Concentration']*dat['amount_int']/(ref_conc[dat['KEGG_ID']]) *(1+Uf)
                        else:
                            ub[dat['KEGG_ID']] = (ub[dat['KEGG_ID']]+dat2['Concentration']*dat['amount_int']/(ref_conc[dat['KEGG_ID']]) *(1+Uf))/2
                        if lb[dat['KEGG_ID']] < DEFAULT_LB:    
                            lb[dat['KEGG_ID']] = DEFAULT_LB

                        if lb[dat['KEGG_ID']] > DEFAULT_UB:
                            lb[dat['KEGG_ID']] = DEFAULT_UB

                        if ub[dat['KEGG_ID']] < DEFAULT_LB:
                            ub[dat['KEGG_ID']] = DEFAULT_LB

                        if ub[dat['KEGG_ID']] > DEFAULT_UB:
                            ub[dat['KEGG_ID']] = DEFAULT_UB

                        pp.bounds.SetBounds(dat['KEGG_ID'], lb[dat['KEGG_ID']], ub[dat['KEGG_ID']])
                        #print(dat['KEGG_ID'], lb, ub)
                        if tym  == 2 and smple == 'A':
                            pp.bounds.SetBounds('C00469', 5.12/46, 5.12/46)
                            pp.bounds.SetBounds('C00033', 2.87/1000, 2.87/1000)
                        elif tym ==5 and smple == 'A':
                            pp.bounds.SetBounds('C00469', 20.21/46, 20.21/46)
                            pp.bounds.SetBounds('C00033', 4.25/1000, 4.25/1000)
                        elif tym ==9 and smple == 'A':
                            pp.bounds.SetBounds('C00469', 40.38/46, 40.38/46)
                            pp.bounds.SetBounds('C00033', 4.39/1000, 4.39/1000)
                        if tym  == 2 and smple == 'B':
                            pp.bounds.SetBounds('C00469', 0.2/46, 0.2/46)
                            pp.bounds.SetBounds('C00033', 3.21/1000, 3.21/1000)
                        elif tym ==5 and smple == 'B':
                            pp.bounds.SetBounds('C00469', 0.47/46, 0.47/46)
                            pp.bounds.SetBounds('C00033', 8.61/1000, 8.61/1000)
                        elif tym ==9 and smple == 'B':
                            pp.bounds.SetBounds('C00469', 0.58/46, 0.58/46)
                            pp.bounds.SetBounds('C00033', 9.21/1000, 9.21/1000)
                        mdf_data = pp.calc_mdf()
                        #print(smple,tym,rep,dat['KEGG_ID'],mdf_data.mdf)
                        atp_ratio=1
                        gtp_ratio=1
                        nadh_ratio=1
                        for c in mdf_data.compound_data:
                            if c.compound in ['C00002']:
                                atp_ratio=atp_ratio*c.concentration
                            elif c.compound in ['C00008']:
                                atp_ratio=atp_ratio/c.concentration
                            elif c.compound in ['C00003']:
                                nadh_ratio=nadh_ratio/c.concentration
                            elif c.compound in ['C00004']:
                                nadh_ratio=nadh_ratio*c.concentration
                            elif c.compound in ['C00035']:
                                gtp_ratio=gtp_ratio/c.concentration
                            elif c.compound in ['C00044']:
                                gtp_ratio=gtp_ratio*c.concentration
                            #print(c.compound, log(mdf_data.model.concentration_bounds.GetBoundTuple(c.compound)), sm,log(all_data[0]['mdf'].model.concentration_bounds.GetBoundTuple(c.compound)))
                        #print('atp_ratio =',atp_ratio)
                        #print('gtp_ratio =',gtp_ratio)
                        #print('nadh_ratio =',nadh_ratio)
                        print(ktaDict[dat['KEGG_ID']],mdf_data.mdf,atp_ratio,gtp_ratio,nadh_ratio)
                        #print("__________________")
                        all_data[sm]['sample']=smple
                        all_data[sm]['timepoint']=tym
                        all_data[sm]['replicate']=rep
                        #all_data[sm][cpd['KEGG_ID']]['metab']=dat['KEGG_ID']
                        all_data[sm]['mdf']=mdf_data
            #except:
                #print("no ref for", dat2['KEGG_ID'])

A 2 1
g6p 3.037842110066581 9.999999999999986 92.37249480015613 0.009999999999999962
g6p 3.0311328495668555 10.000000000000046 92.12269412861441 0.00999999999999996
f6p 3.124169934954459 10.00000000000001 95.64775383986708 0.009999999999999978
f6p 3.1393158493365405 10.00000000000001 96.23424645640613 0.00999999999999996
fdp 1.2373904645672766 9.999999999999993 44.66299090049413 0.009999999999999945
fdp 1.5447925651469205 10.00000000000001 50.56281464442814 0.009999999999999907
mal-l 3.2107686908816575 9.999999999999993 99.04997463072768 0.009999999999999997
mal-l 3.2107686908816575 10.00000000000001 99.04997463072802 0.009999999999999978
dhap -0.7559911264000476 9.999999999999993 19.977213768990975 0.01000000000000005
dhap -0.3114674099625425 9.999999999999993 23.90308325168876 0.009999999999999943
3pg -0.22091051227388547 9.999999999999993 24.79289983840594 0.009999999999999962
3pg -0.0943458135433275 9.999999999999993 26.092302759672332 0.009999999999999997
pep 0.3425638550034904 9.

In [72]:
# leave one out concentration bound at a time to check for infeasible bounds
# Concentrations are scaled with respect to the 2nd time point of sample B, each replicate has its own reference
all_data={}
Uf = 0.1 #Uncertainity factor 
DEFAULT_LB = 1e-6
DEFAULT_UB = 2e-2 #20 mM, which is 2x higher than the 10 mM default bound suggested in the Noor et al. 2014 paper.

ref_conc={}
for i, dat in Met_data.iterrows():
    if dat['Timepoint']==5:
        if dat['KEGG_ID'] not in ref_conc:
            ref_conc[dat['KEGG_ID']]={}
            if not ref_conc[dat['KEGG_ID']]:
                ref_conc[dat['KEGG_ID']]=dat['amount_int']
            else:
                ref_conc[dat['KEGG_ID']]=(ref_conc[dat['KEGG_ID']]+dat['amount_int'])/2
print("Compound name ","Pathway MDF","atp_ratio","gtp_ratio","nadh_ratio")
for i3, dat3 in Met_data_WT.iterrows():
    if dat3['KEGG_ID'] not in ['C00035','C00044','C00022','C00103','C00004','C00005','C00006']:
        print(ktaDict[dat3['KEGG_ID']])
        for sm in [0,1,2,6,7,8]:#range(12): # divide the samples into datasets
            all_data[sm]={}
            if sm < 6:
                smple = 'A'
            else:
                smple = 'B'
            if (sm)%3==0:
                tym = 2
            elif (sm)%3==1:
                tym = 5
            else:
                tym = 9
            if sm<3 or (sm<9 and sm>5):
                rep = 1
            else:
                rep = 2
            #print(smple,tym,rep)
            for i, cpd in Met_bound.iterrows():  # reset bounds at the start of each iteration
                pp.bounds.SetBounds(cpd['KEGG_ID'],cpd['Concentration:Min'],cpd['Concentration:Max'])
            lb={}
            ub={}
            for i2, dat2 in Met_data_WT.iterrows():
                if dat2['KEGG_ID'] not in ['C00035','C00044','C00022','C00103','C00004','C00005','C00006']:
                    #pp.bounds.SetBounds(dat['KEGG_ID'],dat['Concentration']*.50,dat['Concentration']*1.5)
                    try:
                        for i, dat in Met_data.iterrows():
                        # skip cofactors and impose bound on metabolites one at a time    
                            if (dat2['KEGG_ID']==dat['KEGG_ID'])and(dat['Sample']==smple and dat['Timepoint']==tym) and (dat['KEGG_ID'] not in ["C00044","C00035","C00004","C00005","C00006","C00469"]):
                                all_data[sm][i]={}
                                if dat['KEGG_ID'] not in lb:
                                    lb[dat['KEGG_ID']]={}
                                if dat['KEGG_ID'] not in ub:
                                    ub[dat['KEGG_ID']]={}
                                if not lb[dat['KEGG_ID']]:
                                    lb[dat['KEGG_ID']] = dat2['Concentration']*dat['amount_int']/(ref_conc[dat['KEGG_ID']]) *(1-Uf)
                                else:
                                    lb[dat['KEGG_ID']] = (lb[dat['KEGG_ID']]+dat2['Concentration']*dat['amount_int']/(ref_conc[dat['KEGG_ID']]) *(1-Uf))/2
                                if not ub[dat['KEGG_ID']]: 
                                    ub[dat['KEGG_ID']] = dat2['Concentration']*dat['amount_int']/(ref_conc[dat['KEGG_ID']]) *(1+Uf)
                                else:
                                    ub[dat['KEGG_ID']] = (ub[dat['KEGG_ID']]+dat2['Concentration']*dat['amount_int']/(ref_conc[dat['KEGG_ID']]) *(1+Uf))/2
                                if lb[dat['KEGG_ID']] < DEFAULT_LB:    
                                    lb[dat['KEGG_ID']] = DEFAULT_LB

                                if lb[dat['KEGG_ID']] > DEFAULT_UB:
                                    lb[dat['KEGG_ID']] = DEFAULT_UB

                                if ub[dat['KEGG_ID']] < DEFAULT_LB:
                                    ub[dat['KEGG_ID']] = DEFAULT_LB

                                if ub[dat['KEGG_ID']] > DEFAULT_UB:
                                    ub[dat['KEGG_ID']] = DEFAULT_UB

                                #leave one out
                                if (dat2['KEGG_ID']!=dat3['KEGG_ID']):
                                    pp.bounds.SetBounds(dat['KEGG_ID'], lb[dat['KEGG_ID']], ub[dat['KEGG_ID']])
                                #print(dat['KEGG_ID'], lb, ub)
                                if tym  == 2 and smple == 'A':
                                    pp.bounds.SetBounds('C00469', 5.12/46, 5.12/46)
                                    pp.bounds.SetBounds('C00033', 2.87/1000, 2.87/1000)
                                elif tym ==5 and smple == 'A':
                                    pp.bounds.SetBounds('C00469', 20.21/46, 20.21/46)
                                    pp.bounds.SetBounds('C00033', 4.25/1000, 4.25/1000)
                                elif tym ==9 and smple == 'A':
                                    pp.bounds.SetBounds('C00469', 40.38/46, 40.38/46)
                                    pp.bounds.SetBounds('C00033', 4.39/1000, 4.39/1000)
                                if tym  == 2 and smple == 'B':
                                    pp.bounds.SetBounds('C00469', 0.2/46, 0.2/46)
                                    pp.bounds.SetBounds('C00033', 3.21/1000, 3.21/1000)
                                elif tym ==5 and smple == 'B':
                                    pp.bounds.SetBounds('C00469', 0.47/46, 0.47/46)
                                    pp.bounds.SetBounds('C00033', 8.61/1000, 8.61/1000)
                                elif tym ==9 and smple == 'B':
                                    pp.bounds.SetBounds('C00469', 0.58/46, 0.58/46)
                                    pp.bounds.SetBounds('C00033', 9.21/1000, 9.21/1000)
                    except:
                        print("no ref for", dat2['KEGG_ID'])
            mdf_data = pp.calc_mdf()
            #print(smple,tym,rep,dat['KEGG_ID'],mdf_data.mdf)
            atp_ratio=1
            gtp_ratio=1
            nadh_ratio=1
            for c in mdf_data.compound_data:
                if c.compound in ['C00002']:
                    atp_ratio=atp_ratio*c.concentration
                elif c.compound in ['C00008']:
                    atp_ratio=atp_ratio/c.concentration
                elif c.compound in ['C00003']:
                    nadh_ratio=nadh_ratio/c.concentration
                elif c.compound in ['C00004']:
                    nadh_ratio=nadh_ratio*c.concentration
                elif c.compound in ['C00035']:
                    gtp_ratio=gtp_ratio/c.concentration
                elif c.compound in ['C00044']:
                    gtp_ratio=gtp_ratio*c.concentration
                #print(c.compound, c.concentration,(mdf_data.model.concentration_bounds.GetBoundTuple(c.compound)))
            print(smple,tym,rep,mdf_data.mdf,atp_ratio,gtp_ratio,nadh_ratio)

            all_data[sm]['sample']=smple
            all_data[sm]['timepoint']=tym
            all_data[sm]['replicate']=rep

            all_data[sm]['mdf']=mdf_data


Compound name  Pathway MDF atp_ratio gtp_ratio nadh_ratio
g6p
A 2 1 -2.4568736883794813 28.188511368120587 28.344271811562805 0.0022094058505349313
A 5 1 -1.1406493900896102 24.697670699035786 42.244046422588724 0.02358757906937301
A 9 1 -1.1434422965057394 31.787217634622824 54.309083269394755 0.034354617736945446
A 2 2 -2.4568736883794813 28.188511368120587 28.344271811562805 0.0022094058505349313
A 5 2 -1.1406493900896102 24.697670699035786 42.244046422588724 0.02358757906937301
A 9 2 -1.1434422965057394 31.787217634622824 54.309083269394755 0.034354617736945446
B 2 1 -1.8686141702572208 29.30013783218607 37.357411738670386 0.0010718096077589005
B 5 1 -2.7479973535763236 55.07240946925464 49.237635781795255 0.0015186385769057889
B 9 1 -1.075153385075243 2.9684267529494943 10.00000000000001 0.001728755643325183
B 2 2 -1.8686141702572208 29.30013783218607 37.357411738670386 0.0010718096077589005
B 5 2 -2.7479973535763236 55.07240946925464 49.237635781795255 0.0015186385769057889
B 9 2

In [140]:
# leave one out concentration bound at a time to check for infeasible bounds
# Concentrations are scaled with respect to the 2nd time point of sample B, each replicate has its own reference
all_data={}
Uf = 0.3 #Uncertainity factor 
DEFAULT_LB = 1e-6
DEFAULT_UB = 2e-2 #20 mM, which is 2x higher than the 10 mM default bound suggested in the Noor et al. 2014 paper.

ref_conc={}
for i, dat in Met_data.iterrows():
    if dat['Timepoint']==5 and dat['Sample']=='B':
        if dat['KEGG_ID'] not in ref_conc:
            ref_conc[dat['KEGG_ID']]={}
        if dat['Replicate']==1:
            ref_conc[dat['KEGG_ID']]=dat['amount_int']
        else:
            ref_conc[dat['KEGG_ID']]=(ref_conc[dat['KEGG_ID']]+dat['amount_int'])/2
print("Compound name ","Pathway MDF","atp_ratio","gtp_ratio","nadh_ratio")
chk_list=[]
for i4, dat4 in Met_data_WT.iterrows():
    if dat4['KEGG_ID'] not in ['C00002','C00008','C00003','C00020','C00035','C00044','C00022','C00103','C00004','C00005','C00006'] :
        chk_list.append(dat4['KEGG_ID'])
        for i3, dat3 in Met_data_WT.iterrows():
            if dat3['KEGG_ID'] not in ['C00002','C00008','C00003','C00020','C00035','C00044','C00022','C00103','C00004','C00005','C00006'] and dat3['KEGG_ID'] not in chk_list :
                print(ktaDict[dat4['KEGG_ID']],ktaDict[dat3['KEGG_ID']])
                for sm in [0,1,2,6,7,8]:#range(12): # divide the samples into datasets
                    all_data[sm]={}
                    if sm < 6:
                        smple = 'A'
                    else:
                        smple = 'B'
                    if (sm)%3==0:
                        tym = 2
                    elif (sm)%3==1:
                        tym = 5
                    else:
                        tym = 9
                    if sm<3 or (sm<9 and sm>5):
                        rep = 1
                    else:
                        rep = 2
                    #print(smple,tym,rep)
                    for i, cpd in Met_bound.iterrows():  # reset bounds at the start of each iteration
                        pp.bounds.SetBounds(cpd['KEGG_ID'],cpd['Concentration:Min'],cpd['Concentration:Max'])
                    lb={}
                    ub={}
                    for i2, dat2 in Met_data_WT.iterrows():
                        if dat2['KEGG_ID'] not in ['C00002','C00008','C00003','C00020','C00035','C00044','C00022','C00103','C00004','C00005','C00006']:
                            #pp.bounds.SetBounds(dat['KEGG_ID'],dat['Concentration']*.50,dat['Concentration']*1.5)
                            try:
                                for i, dat in Met_data.iterrows():
                                # skip cofactors and impose bound on metabolites one at a time    
                                    if (dat2['KEGG_ID']==dat['KEGG_ID'])and(dat['Sample']==smple and dat['Timepoint']==tym) and (dat['KEGG_ID'] not in ['C00002','C00008','C00003','C00020',"C00044","C00035","C00004","C00005","C00006","C00469"]):
                                        all_data[sm][i]={}
                                        if dat['KEGG_ID'] not in lb:
                                            lb[dat['KEGG_ID']]={}
                                        if dat['KEGG_ID'] not in ub:
                                            ub[dat['KEGG_ID']]={}
                                        if not lb[dat['KEGG_ID']]:
                                            lb[dat['KEGG_ID']] = dat2['Concentration']*dat['amount_int']/(ref_conc[dat['KEGG_ID']]) *(1-Uf)
                                        else:
                                            lb[dat['KEGG_ID']] = (lb[dat['KEGG_ID']]+dat2['Concentration']*dat['amount_int']/(ref_conc[dat['KEGG_ID']]) *(1-Uf))/2
                                        if not ub[dat['KEGG_ID']]: 
                                            ub[dat['KEGG_ID']] = dat2['Concentration']*dat['amount_int']/(ref_conc[dat['KEGG_ID']]) *(1+Uf)
                                        else:
                                            ub[dat['KEGG_ID']] = (ub[dat['KEGG_ID']]+dat2['Concentration']*dat['amount_int']/(ref_conc[dat['KEGG_ID']]) *(1+Uf))/2
                                        if lb[dat['KEGG_ID']] < DEFAULT_LB:    
                                            lb[dat['KEGG_ID']] = DEFAULT_LB

                                        if lb[dat['KEGG_ID']] > DEFAULT_UB:
                                            lb[dat['KEGG_ID']] = DEFAULT_UB

                                        if ub[dat['KEGG_ID']] < DEFAULT_LB:
                                            ub[dat['KEGG_ID']] = DEFAULT_LB

                                        if ub[dat['KEGG_ID']] > DEFAULT_UB:
                                            ub[dat['KEGG_ID']] = DEFAULT_UB

                                        #leave two out
                                        if (dat2['KEGG_ID']!=dat3['KEGG_ID'] and dat2['KEGG_ID']!=dat4['KEGG_ID']):
                                            pp.bounds.SetBounds(dat['KEGG_ID'], lb[dat['KEGG_ID']], ub[dat['KEGG_ID']])
                                        #print(dat['KEGG_ID'], lb, ub)
                                        if tym  == 2 and smple == 'A':
                                            pp.bounds.SetBounds('C00469', 5.12/46, 5.12/46)
                                            pp.bounds.SetBounds('C00033', 2.87/1000, 2.87/1000)
                                        elif tym ==5 and smple == 'A':
                                            pp.bounds.SetBounds('C00469', 20.21/46, 20.21/46)
                                            pp.bounds.SetBounds('C00033', 4.25/1000, 4.25/1000)
                                        elif tym ==9 and smple == 'A':
                                            pp.bounds.SetBounds('C00469', 40.38/46, 40.38/46)
                                            pp.bounds.SetBounds('C00033', 4.39/1000, 4.39/1000)
                                        if tym  == 2 and smple == 'B':
                                            pp.bounds.SetBounds('C00469', 0.2/46, 0.2/46)
                                            pp.bounds.SetBounds('C00033', 3.21/1000, 3.21/1000)
                                        elif tym ==5 and smple == 'B':
                                            pp.bounds.SetBounds('C00469', 0.47/46, 0.47/46)
                                            pp.bounds.SetBounds('C00033', 8.61/1000, 8.61/1000)
                                        elif tym ==9 and smple == 'B':
                                            pp.bounds.SetBounds('C00469', 0.58/46, 0.58/46)
                                            pp.bounds.SetBounds('C00033', 9.21/1000, 9.21/1000)
                            except:
                                print("no ref for", dat2['KEGG_ID'])
                    if 1:
                        mdf_data = pp.calc_mdf()
                        #print(smple,tym,rep,dat['KEGG_ID'],mdf_data.mdf)
                        atp_ratio=1
                        gtp_ratio=1
                        nadh_ratio=1
                        for c in mdf_data.compound_data:
                            if c.compound in ['C00002']:
                                atp_ratio=atp_ratio*c.concentration
                            elif c.compound in ['C00008']:
                                atp_ratio=atp_ratio/c.concentration
                            elif c.compound in ['C00003']:
                                nadh_ratio=nadh_ratio/c.concentration
                            elif c.compound in ['C00004']:
                                nadh_ratio=nadh_ratio*c.concentration
                            elif c.compound in ['C00035']:
                                gtp_ratio=gtp_ratio/c.concentration
                            elif c.compound in ['C00044']:
                                gtp_ratio=gtp_ratio*c.concentration
                            #print(c.compound, c.concentration,(mdf_data.model.concentration_bounds.GetBoundTuple(c.compound)))
                        print(smple,tym,rep,mdf_data.mdf,atp_ratio,gtp_ratio,nadh_ratio)

                        all_data[sm]['sample']=smple
                        all_data[sm]['timepoint']=tym
                        all_data[sm]['replicate']=rep

                        all_data[sm]['mdf']=mdf_data


Compound name  Pathway MDF atp_ratio gtp_ratio nadh_ratio
g6p f6p
no ref for C00002
A 2 1 1.2160392490227778 2.258368342087103 10.00000000000001 0.009140502981942614
no ref for C00002
A 5 1 1.1441371390306223 1.8211713800565268 13.017168555950873 0.0636586652740141
no ref for C00002
A 9 1 0.9789074711607775 1.3073134103417956 18.34360228216724 0.0868333004411252
no ref for C00002
B 2 1 1.6712876488180548 1.879298978859343 10.00000000000001 0.004800611640112116
B 5 1 0.6839982393665593 7.256833353491493 25.923448010095793 0.006512066861259512
no ref for C00002
B 9 1 1.4628070912094628 2.0442768256533337 9.999999999999993 0.005640519910189541
g6p fdp
no ref for C00002
A 2 1 1.2160392490227707 2.258368342087109 9.999999999999995 0.009140502981942647
no ref for C00002
A 5 1 1.28529520407508 1.5350463963335004 12.29627096974967 0.06739080311073872
no ref for C00002
A 9 1 0.9303787017280349 1.3331719097561523 18.70643649247848 0.08514906239787515
no ref for C00002
B 2 1 1.6712876488180477 1.

KeyboardInterrupt: 

In [163]:
# impose only one concentration bound at a time to check for infeasible bounds
all_data={}
Uf = 0.00 #Uncertainity factor 
DEFAULT_LB = 1e-6
DEFAULT_UB = 2e-2 #20 mM, which is 2x higher than the 10 mM default bound suggested in the Noor et al. 2014 paper.

ref_conc={}
for i, dat in Met_data.iterrows():
    if dat['Timepoint']==5 and dat['Sample']=='B':
        if dat['KEGG_ID'] not in ref_conc:
            ref_conc[dat['KEGG_ID']]={}
        if dat['Replicate']==1:
            ref_conc[dat['KEGG_ID']]=dat['amount']
        else:
            ref_conc[dat['KEGG_ID']]=(ref_conc[dat['KEGG_ID']]+dat['amount'])/2
for sm in [0,1,2,6,7,8]:#range(12): # divide the samples into datasets
    all_data[sm]={}
    if sm < 6:
        smple = 'A'
    else:
        smple = 'B'
    if (sm)%3==0:
        tym = 2
    elif (sm)%3==1:
        tym = 5
    else:
        tym = 9
    if sm<3 or (sm<9 and sm>5):
        rep = 1
    else:
        rep = 2
    #print(smple,tym,rep)
    for i, cpd in Met_bound.iterrows():  # reset bounds at the start of each iteration
        pp.bounds.SetBounds(cpd['KEGG_ID'],cpd['Concentration:Min'],cpd['Concentration:Max'] )
    for i2, dat2 in Met_data_WT.iterrows():
        if dat2['KEGG_ID'] not in ['C00044','C00035','C00002','C00004','C00005']:
            #pp.bounds.SetBounds(dat['KEGG_ID'],dat['Concentration']*.50,dat['Concentration']*1.5)
            lb={}
            ub={}
            for i, dat in Met_data.iterrows():
                try:
                    if (dat2['KEGG_ID']==dat['KEGG_ID'])and(dat['Sample']==smple and dat['Timepoint']==tym) and (dat['KEGG_ID'] not in ['C00035',"C00044","C00002","C00004","C00005"]):
                        all_data[sm][i]={}
                        if dat['KEGG_ID'] not in lb:
                            lb[dat['KEGG_ID']]={}
                        if dat['KEGG_ID'] not in ub:
                            ub[dat['KEGG_ID']]={}
                        if not lb[dat['KEGG_ID']]:
                            lb[dat['KEGG_ID']] = dat['amount']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1-Uf)
                        else:
                            lb[dat['KEGG_ID']] = (lb[dat['KEGG_ID']]+dat['amount']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1-Uf))/2
                            if lb[dat['KEGG_ID']] < DEFAULT_LB:    
                                lb[dat['KEGG_ID']] = DEFAULT_LB

                            if lb[dat['KEGG_ID']] > DEFAULT_UB:
                                lb[dat['KEGG_ID']] = DEFAULT_UB
                        if not ub[dat['KEGG_ID']]: 
                            ub[dat['KEGG_ID']] = dat['amount']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1+Uf)
                        else:
                            ub[dat['KEGG_ID']] = (ub[dat['KEGG_ID']]+dat['amount']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1+Uf))/2
                            if ub[dat['KEGG_ID']] < DEFAULT_LB:
                                ub[dat['KEGG_ID']] = DEFAULT_LB

                            if ub[dat['KEGG_ID']] > DEFAULT_UB:
                                ub[dat['KEGG_ID']] = DEFAULT_UB
                        #if dat['KEGG_ID']=='C00085':
                        #print(dat['KEGG_ID'],ref_conc[dat['KEGG_ID']],dat2['Concentration'],lb[dat['KEGG_ID']],ub[dat['KEGG_ID']],dat['amount_int'])
                        pp.bounds.SetBounds(dat['KEGG_ID'], lb[dat['KEGG_ID']], ub[dat['KEGG_ID']])
                        #print(dat['KEGG_ID'], lb[dat['KEGG_ID']], ub[dat['KEGG_ID']])
                except:
                    print("ref not found for",dat['KEGG_ID'])
                    #print(dat['KEGG_ID'], lb, ub)
                if tym  == 2 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 5.12/46, 5.12/46)
                    pp.bounds.SetBounds('C00033', 2.87/1000, 2.87/1000)
                elif tym ==5 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 20.21/46, 20.21/46)
                    pp.bounds.SetBounds('C00033', 4.25/1000, 4.25/1000)
                elif tym ==9 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 40.38/46, 40.38/46)
                    pp.bounds.SetBounds('C00033', 4.39/1000, 4.39/1000)
                if tym  == 2 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.2/46, 0.2/46)
                    pp.bounds.SetBounds('C00033', 3.21/1000, 3.21/1000)
                elif tym ==5 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.47/46, 0.47/46)
                    pp.bounds.SetBounds('C00033', 8.61/1000, 8.61/1000)
                elif tym ==9 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.58/46, 0.58/46)
                    pp.bounds.SetBounds('C00033', 9.21/1000, 9.21/1000)
    mdf_data = pp.calc_mdf()
    #print(smple,tym,rep,dat['KEGG_ID'],mdf_data.mdf)
    atp_ratio=1
    gtp_ratio=1
    nadh_ratio=1
    for c in mdf_data.compound_data:
        if c.compound in ['C00002']:
            atp_ratio=atp_ratio*c.concentration
        elif c.compound in ['C00008']:
            atp_ratio=atp_ratio/c.concentration
        elif c.compound in ['C00003']:
            nadh_ratio=nadh_ratio/c.concentration
        elif c.compound in ['C00004']:
            nadh_ratio=nadh_ratio*c.concentration
        elif c.compound in ['C00035']:
            gtp_ratio=gtp_ratio/c.concentration
        elif c.compound in ['C00044']:
            gtp_ratio=gtp_ratio*c.concentration
        #print(c.compound, c.concentration,(mdf_data.model.concentration_bounds.GetBoundTuple(c.compound)),c.shadow_price)
    print('atp_ratio =',atp_ratio)
    print('gtp_ratio =',gtp_ratio)
    print('nadh_ratio =',nadh_ratio)
    print(smple,tym,rep,mdf_data.mdf)
    print("__________________")
    all_data[sm]['sample']=smple
    all_data[sm]['timepoint']=tym
    all_data[sm]['replicate']=rep
    #all_data[sm][cpd['KEGG_ID']]['metab']=dat['KEGG_ID']
    all_data[sm]['mdf']=mdf_data
    #except:
    #print("no ref for", dat2['KEGG_ID'])

atp_ratio = 2.6815346166950125
gtp_ratio = 10.078087276938458
nadh_ratio = 0.009228268976467911
A 2 1 0.8097871145709625
__________________
atp_ratio = 7.307495491814929
gtp_ratio = 23.064691674857514
nadh_ratio = 0.06365866527401343
A 5 1 0.3772631653874825
__________________
atp_ratio = 1.8414346852636907
gtp_ratio = 33.15935499895112
nadh_ratio = 0.08803474793245816
A 9 1 0.2460795683801429
__________________
atp_ratio = 4.306535943079639
gtp_ratio = 17.826867303849767
nadh_ratio = 0.0050893470057076185
B 2 1 1.0491224187779338
__________________
atp_ratio = 9.433883359538926
gtp_ratio = 26.557907477450453
nadh_ratio = 0.006993575100735794
B 5 1 0.09386572432115159
__________________
atp_ratio = 8.465054215078435
gtp_ratio = 30.385569630481953
nadh_ratio = 0.005167562594539449
B 9 1 0.6959331175663301
__________________


In [160]:
# impose only one concentration bound at a time to check for infeasible bounds
all_data={}
Uf = 0.05 #Uncertainity factor 
DEFAULT_LB = 1e-6
DEFAULT_UB = 2e-2 #20 mM, which is 2x higher than the 10 mM default bound suggested in the Noor et al. 2014 paper.

ref_conc={}
for i, dat in Met_data.iterrows():
    if dat['Timepoint']==5 and dat['Sample']=='B':
        if dat['KEGG_ID'] not in ref_conc:
            ref_conc[dat['KEGG_ID']]={}
        if dat['Replicate']==1:
            ref_conc[dat['KEGG_ID']]=dat['amount_int']
        else:
            ref_conc[dat['KEGG_ID']]=(ref_conc[dat['KEGG_ID']]+dat['amount_int'])/2
for sm in [0,1,2,6,7,8]:#range(12): # divide the samples into datasets
    all_data[sm]={}
    if sm < 6:
        smple = 'A'
    else:
        smple = 'B'
    if (sm)%3==0:
        tym = 2
    elif (sm)%3==1:
        tym = 5
    else:
        tym = 9
    if sm<3 or (sm<9 and sm>5):
        rep = 1
    else:
        rep = 2
    #print(smple,tym,rep)
    for i, cpd in Met_bound.iterrows():  # reset bounds at the start of each iteration
        pp.bounds.SetBounds(cpd['KEGG_ID'],cpd['Concentration:Min'],cpd['Concentration:Max'] )
    for i2, dat2 in Met_data_WT.iterrows():
        if dat2['KEGG_ID'] not in ['C00044','C00035','C00002','C00004','C00005']:
            #pp.bounds.SetBounds(dat['KEGG_ID'],dat['Concentration']*.50,dat['Concentration']*1.5)
            lb={}
            ub={}
            for i, dat in Met_data.iterrows():
                try:
                    if (dat2['KEGG_ID']==dat['KEGG_ID'])and(dat['Sample']==smple and dat['Timepoint']==tym) and (dat['KEGG_ID'] not in ['C00035',"C00044","C00002","C00004","C00005"]):
                        all_data[sm][i]={}
                        if dat['KEGG_ID'] not in lb:
                            lb[dat['KEGG_ID']]={}
                        if dat['KEGG_ID'] not in ub:
                            ub[dat['KEGG_ID']]={}
                        if not lb[dat['KEGG_ID']]:
                            lb[dat['KEGG_ID']] = dat['amount_int']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1-Uf)
                        else:
                            lb[dat['KEGG_ID']] = (lb[dat['KEGG_ID']]+dat['amount_int']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1-Uf))/2
                            if lb[dat['KEGG_ID']] < DEFAULT_LB:    
                                lb[dat['KEGG_ID']] = DEFAULT_LB

                            if lb[dat['KEGG_ID']] > DEFAULT_UB:
                                lb[dat['KEGG_ID']] = DEFAULT_UB
                        if not ub[dat['KEGG_ID']]: 
                            ub[dat['KEGG_ID']] = dat['amount_int']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1+Uf)
                        else:
                            ub[dat['KEGG_ID']] = (ub[dat['KEGG_ID']]+dat['amount_int']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1+Uf))/2
                            if ub[dat['KEGG_ID']] < DEFAULT_LB:
                                ub[dat['KEGG_ID']] = DEFAULT_LB

                            if ub[dat['KEGG_ID']] > DEFAULT_UB:
                                ub[dat['KEGG_ID']] = DEFAULT_UB
                        #if dat['KEGG_ID']=='C00085':
                        #print(dat['KEGG_ID'],ref_conc[dat['KEGG_ID']],dat2['Concentration'],lb[dat['KEGG_ID']],ub[dat['KEGG_ID']],dat['amount_int'])
                        pp.bounds.SetBounds(dat['KEGG_ID'], lb[dat['KEGG_ID']], ub[dat['KEGG_ID']])
                        #print(dat['KEGG_ID'], lb[dat['KEGG_ID']], ub[dat['KEGG_ID']])
                except:
                    print("ref not found for",dat['KEGG_ID'])
                    #print(dat['KEGG_ID'], lb, ub)
                if tym  == 2 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 5.12/46, 5.12/46)
                    pp.bounds.SetBounds('C00033', 2.87/1000, 2.87/1000)
                elif tym ==5 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 20.21/46, 20.21/46)
                    pp.bounds.SetBounds('C00033', 4.25/1000, 4.25/1000)
                elif tym ==9 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 40.38/46, 40.38/46)
                    pp.bounds.SetBounds('C00033', 4.39/1000, 4.39/1000)
                if tym  == 2 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.2/46, 0.2/46)
                    pp.bounds.SetBounds('C00033', 3.21/1000, 3.21/1000)
                elif tym ==5 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.47/46, 0.47/46)
                    pp.bounds.SetBounds('C00033', 8.61/1000, 8.61/1000)
                elif tym ==9 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.58/46, 0.58/46)
                    pp.bounds.SetBounds('C00033', 9.21/1000, 9.21/1000)
    mdf_data = pp.calc_mdf()
    #print(smple,tym,rep,dat['KEGG_ID'],mdf_data.mdf)
    atp_ratio=1
    gtp_ratio=1
    nadh_ratio=1
    for c in mdf_data.compound_data:
        if c.compound in ['C00002']:
            atp_ratio=atp_ratio*c.concentration
        elif c.compound in ['C00008']:
            atp_ratio=atp_ratio/c.concentration
        elif c.compound in ['C00003']:
            nadh_ratio=nadh_ratio/c.concentration
        elif c.compound in ['C00004']:
            nadh_ratio=nadh_ratio*c.concentration
        elif c.compound in ['C00035']:
            gtp_ratio=gtp_ratio/c.concentration
        elif c.compound in ['C00044']:
            gtp_ratio=gtp_ratio*c.concentration
        #print(c.compound, c.concentration,(mdf_data.model.concentration_bounds.GetBoundTuple(c.compound)),c.shadow_price)
    print('atp_ratio =',atp_ratio)
    print('gtp_ratio =',gtp_ratio)
    print('nadh_ratio =',nadh_ratio)
    print(smple,tym,rep,mdf_data.mdf)
    print("__________________")
    all_data[sm]['sample']=smple
    all_data[sm]['timepoint']=tym
    all_data[sm]['replicate']=rep
    #all_data[sm][cpd['KEGG_ID']]['metab']=dat['KEGG_ID']
    all_data[sm]['mdf']=mdf_data
    #except:
    #print("no ref for", dat2['KEGG_ID'])

atp_ratio = 2.573575584616294
gtp_ratio = 9.999999999999977
nadh_ratio = 0.009301627822090923
A 2 1 0.8923283529425277
__________________
atp_ratio = 6.958476109789918
gtp_ratio = 23.09011440483575
nadh_ratio = 0.06365866527401427
A 5 1 0.5012478799536915
__________________
atp_ratio = 1.9471834910209727
gtp_ratio = 29.20642389104715
nadh_ratio = 0.08752586518871523
A 9 1 0.3557008687383245
__________________
atp_ratio = 3.525304932997191
gtp_ratio = 15.341806155366475
nadh_ratio = 0.00508934700570761
B 2 1 1.1731071333441463
__________________
atp_ratio = 8.984650818608516
gtp_ratio = 26.31977197883736
nadh_ratio = 0.006922197146469236
B 5 1 0.19243331379798434
__________________
atp_ratio = 6.929443473114145
gtp_ratio = 26.149828303853155
nadh_ratio = 0.005167562594539436
B 9 1 0.8199178321325533
__________________


In [173]:
# average using peakareatop
# impose only one concentration bound at a time to check for infeasible bounds
all_data={}
Uf = 0.0 #Uncertainity factor 
DEFAULT_LB = 1e-6
DEFAULT_UB = 2e-2 #20 mM, which is 2x higher than the 10 mM default bound suggested in the Noor et al. 2014 paper.

ref_conc={}
for i, dat in Met_data.iterrows():
    if dat['Timepoint']==5 and dat['Sample']=='B':
        if dat['KEGG_ID'] not in ref_conc:
            ref_conc[dat['KEGG_ID']]={}
        if dat['Replicate']==1:
            ref_conc[dat['KEGG_ID']]=dat['peakAreaTop']
        else:
            ref_conc[dat['KEGG_ID']]=(ref_conc[dat['KEGG_ID']]+dat['peakAreaTop'])/2
for sm in [0,1,2,6,7,8]:#range(12): # divide the samples into datasets
    all_data[sm]={}
    if sm < 6:
        smple = 'A'
    else:
        smple = 'B'
    if (sm)%3==0:
        tym = 2
    elif (sm)%3==1:
        tym = 5
    else:
        tym = 9
    if sm<3 or (sm<9 and sm>5):
        rep = 1
    else:
        rep = 2
    #print(smple,tym,rep)
    for i, cpd in Met_bound.iterrows():  # reset bounds at the start of each iteration
        pp.bounds.SetBounds(cpd['KEGG_ID'],cpd['Concentration:Min'],cpd['Concentration:Max'] )
    for i2, dat2 in Met_data_WT.iterrows():
        if dat2['KEGG_ID'] not in ['C00044','C00035','C00002','C00004','C00005']:
            #pp.bounds.SetBounds(dat['KEGG_ID'],dat['Concentration']*.50,dat['Concentration']*1.5)
            lb={}
            ub={}
            for i, dat in Met_data.iterrows():
                try:
                    if (dat2['KEGG_ID']==dat['KEGG_ID'])and(dat['Sample']==smple and dat['Timepoint']==tym) and (dat['KEGG_ID'] not in ['C00035',"C00044","C00002","C00004","C00005"]):
                        all_data[sm][i]={}
                        if dat['KEGG_ID'] not in lb:
                            lb[dat['KEGG_ID']]={}
                        if dat['KEGG_ID'] not in ub:
                            ub[dat['KEGG_ID']]={}
                        if not lb[dat['KEGG_ID']]:
                            lb[dat['KEGG_ID']] = dat['peakAreaTop']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1-Uf)
                        else:
                            lb[dat['KEGG_ID']] = (lb[dat['KEGG_ID']]+dat['peakAreaTop']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1-Uf))/2
                            if lb[dat['KEGG_ID']] < DEFAULT_LB:    
                                lb[dat['KEGG_ID']] = DEFAULT_LB

                            if lb[dat['KEGG_ID']] > DEFAULT_UB:
                                lb[dat['KEGG_ID']] = DEFAULT_UB
                        if not ub[dat['KEGG_ID']]: 
                            ub[dat['KEGG_ID']] = dat['peakAreaTop']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1+Uf)
                        else:
                            ub[dat['KEGG_ID']] = (ub[dat['KEGG_ID']]+dat['peakAreaTop']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1+Uf))/2
                            if ub[dat['KEGG_ID']] < DEFAULT_LB:
                                ub[dat['KEGG_ID']] = DEFAULT_LB

                            if ub[dat['KEGG_ID']] > DEFAULT_UB:
                                ub[dat['KEGG_ID']] = DEFAULT_UB
                        #if dat['KEGG_ID']=='C00085':
                        #print(dat['KEGG_ID'],ref_conc[dat['KEGG_ID']],dat2['Concentration'],lb[dat['KEGG_ID']],ub[dat['KEGG_ID']],dat['amount_int'])
                        pp.bounds.SetBounds(dat['KEGG_ID'], lb[dat['KEGG_ID']], ub[dat['KEGG_ID']])
                        #print(dat['KEGG_ID'], lb[dat['KEGG_ID']], ub[dat['KEGG_ID']])
                except:
                    print("ref not found for",dat['KEGG_ID'])
                    #print(dat['KEGG_ID'], lb, ub)
                if tym  == 2 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 5.12/46, 5.12/46)
                    pp.bounds.SetBounds('C00033', 2.87/1000, 2.87/1000)
                elif tym ==5 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 20.21/46, 20.21/46)
                    pp.bounds.SetBounds('C00033', 4.25/1000, 4.25/1000)
                elif tym ==9 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 40.38/46, 40.38/46)
                    pp.bounds.SetBounds('C00033', 4.39/1000, 4.39/1000)
                if tym  == 2 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.2/46, 0.2/46)
                    pp.bounds.SetBounds('C00033', 3.21/1000, 3.21/1000)
                elif tym ==5 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.47/46, 0.47/46)
                    pp.bounds.SetBounds('C00033', 8.61/1000, 8.61/1000)
                elif tym ==9 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.58/46, 0.58/46)
                    pp.bounds.SetBounds('C00033', 9.21/1000, 9.21/1000)
    mdf_data = pp.calc_mdf()
    #print(smple,tym,rep,dat['KEGG_ID'],mdf_data.mdf)
    atp_ratio=1
    gtp_ratio=1
    nadh_ratio=1
    for c in mdf_data.compound_data:
        if c.compound in ['C00002']:
            atp_ratio=atp_ratio*c.concentration
        elif c.compound in ['C00008']:
            atp_ratio=atp_ratio/c.concentration
        elif c.compound in ['C00003']:
            nadh_ratio=nadh_ratio/c.concentration
        elif c.compound in ['C00004']:
            nadh_ratio=nadh_ratio*c.concentration
        elif c.compound in ['C00035']:
            gtp_ratio=gtp_ratio/c.concentration
        elif c.compound in ['C00044']:
            gtp_ratio=gtp_ratio*c.concentration
        #print(c.compound, c.concentration,(mdf_data.model.concentration_bounds.GetBoundTuple(c.compound)),c.shadow_price)
    #print('atp_ratio =',atp_ratio)
    #print('gtp_ratio =',gtp_ratio)
    #print('nadh_ratio =',nadh_ratio)
    print(smple,tym,rep,mdf_data.mdf)
    #print("__________________")
    all_data[sm]['sample']=smple
    all_data[sm]['timepoint']=tym
    all_data[sm]['replicate']=rep
    #all_data[sm][cpd['KEGG_ID']]['metab']=dat['KEGG_ID']
    all_data[sm]['mdf']=mdf_data
    #except:
    #print("no ref for", dat2['KEGG_ID'])

A 2 1 0.7946512530892811
A 5 1 0.2847407938397133
A 9 1 0.1925940712903511
B 2 1 1.0016204419114771
B 5 1 0.09386572432114804
B 9 1 0.6809372394264024


In [166]:
# average using peakareatop
# impose only one concentration bound at a time to check for infeasible bounds
all_data={}
Uf = 0.0 #Uncertainity factor 
DEFAULT_LB = 1e-6
DEFAULT_UB = 2e-2 #20 mM, which is 2x higher than the 10 mM default bound suggested in the Noor et al. 2014 paper.

ref_conc={}
for i, dat in Met_data.iterrows():
    if dat['Timepoint']==5 and dat['Sample']=='B':
        if dat['KEGG_ID'] not in ref_conc:
            ref_conc[dat['KEGG_ID']]={}
        if dat['Replicate']==1:
            ref_conc[dat['KEGG_ID']]=dat['amount_int']
        else:
            ref_conc[dat['KEGG_ID']]=(ref_conc[dat['KEGG_ID']]+dat['amount_int'])/2
for sm in [0,1,2,6,7,8]:#range(12): # divide the samples into datasets
    all_data[sm]={}
    if sm < 6:
        smple = 'A'
    else:
        smple = 'B'
    if (sm)%3==0:
        tym = 2
    elif (sm)%3==1:
        tym = 5
    else:
        tym = 9
    if sm<3 or (sm<9 and sm>5):
        rep = 1
    else:
        rep = 2
    #print(smple,tym,rep)
    for i, cpd in Met_bound.iterrows():  # reset bounds at the start of each iteration
        pp.bounds.SetBounds(cpd['KEGG_ID'],cpd['Concentration:Min'],cpd['Concentration:Max'] )
    for i2, dat2 in Met_data_WT.iterrows():
        if dat2['KEGG_ID'] not in ['C00044','C00035','C00002','C00004','C00005']:
            #pp.bounds.SetBounds(dat['KEGG_ID'],dat['Concentration']*.50,dat['Concentration']*1.5)
            lb={}
            ub={}
            for i, dat in Met_data.iterrows():
                try:
                    if (dat2['KEGG_ID']==dat['KEGG_ID'])and(dat['Sample']==smple and dat['Timepoint']==tym) and (dat['KEGG_ID'] not in ['C00035',"C00044","C00002","C00004","C00005"]):
                        all_data[sm][i]={}
                        if dat['KEGG_ID'] not in lb:
                            lb[dat['KEGG_ID']]={}
                        if dat['KEGG_ID'] not in ub:
                            ub[dat['KEGG_ID']]={}
                        if not lb[dat['KEGG_ID']]:
                            lb[dat['KEGG_ID']] = dat['amount_int']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1-Uf)
                        else:
                            lb[dat['KEGG_ID']] = (lb[dat['KEGG_ID']]+dat['amount_int']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1-Uf))/2
                            #if lb[dat['KEGG_ID']] < DEFAULT_LB:    
                                #lb[dat['KEGG_ID']] = DEFAULT_LB

                            #if lb[dat['KEGG_ID']] > DEFAULT_UB:
                                #lb[dat['KEGG_ID']] = DEFAULT_UB
                        if not ub[dat['KEGG_ID']]: 
                            ub[dat['KEGG_ID']] = dat['amount_int']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1+Uf)
                        else:
                            ub[dat['KEGG_ID']] = (ub[dat['KEGG_ID']]+dat['amount_int']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1+Uf))/2
                            #if ub[dat['KEGG_ID']] < DEFAULT_LB:
                                #ub[dat['KEGG_ID']] = DEFAULT_LB

                            #if ub[dat['KEGG_ID']] > DEFAULT_UB:
                                #ub[dat['KEGG_ID']] = DEFAULT_UB
                        #if dat['KEGG_ID']=='C00085':
                        #print(dat['KEGG_ID'],ref_conc[dat['KEGG_ID']],dat2['Concentration'],lb[dat['KEGG_ID']],ub[dat['KEGG_ID']],dat['amount_int'])
                        pp.bounds.SetBounds(dat['KEGG_ID'], lb[dat['KEGG_ID']], ub[dat['KEGG_ID']])
                        #print(dat['KEGG_ID'], lb[dat['KEGG_ID']], ub[dat['KEGG_ID']])
                except:
                    print("ref not found for",dat['KEGG_ID'])
                    #print(dat['KEGG_ID'], lb, ub)
                if tym  == 2 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 5.12/46, 5.12/46)
                    pp.bounds.SetBounds('C00033', 2.87/1000, 2.87/1000)
                elif tym ==5 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 20.21/46, 20.21/46)
                    pp.bounds.SetBounds('C00033', 4.25/1000, 4.25/1000)
                elif tym ==9 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 40.38/46, 40.38/46)
                    pp.bounds.SetBounds('C00033', 4.39/1000, 4.39/1000)
                if tym  == 2 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.2/46, 0.2/46)
                    pp.bounds.SetBounds('C00033', 3.21/1000, 3.21/1000)
                elif tym ==5 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.47/46, 0.47/46)
                    pp.bounds.SetBounds('C00033', 8.61/1000, 8.61/1000)
                elif tym ==9 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.58/46, 0.58/46)
                    pp.bounds.SetBounds('C00033', 9.21/1000, 9.21/1000)
    mdf_data = pp.calc_mdf()
    #print(smple,tym,rep,dat['KEGG_ID'],mdf_data.mdf)
    atp_ratio=1
    gtp_ratio=1
    nadh_ratio=1
    for c in mdf_data.compound_data:
        if c.compound in ['C00002']:
            atp_ratio=atp_ratio*c.concentration
        elif c.compound in ['C00008']:
            atp_ratio=atp_ratio/c.concentration
        elif c.compound in ['C00003']:
            nadh_ratio=nadh_ratio/c.concentration
        elif c.compound in ['C00004']:
            nadh_ratio=nadh_ratio*c.concentration
        elif c.compound in ['C00035']:
            gtp_ratio=gtp_ratio/c.concentration
        elif c.compound in ['C00044']:
            gtp_ratio=gtp_ratio*c.concentration
        #print(c.compound, c.concentration,(mdf_data.model.concentration_bounds.GetBoundTuple(c.compound)),c.shadow_price)
    #print('atp_ratio =',atp_ratio)
    #print('gtp_ratio =',gtp_ratio)
    #print('nadh_ratio =',nadh_ratio)
    print(smple,tym,rep,mdf_data.mdf)
    #print("__________________")
    all_data[sm]['sample']=smple
    all_data[sm]['timepoint']=tym
    all_data[sm]['replicate']=rep
    #all_data[sm][cpd['KEGG_ID']]['metab']=dat['KEGG_ID']
    all_data[sm]['mdf']=mdf_data
    #except:
    #print("no ref for", dat2['KEGG_ID'])

A 2 1 0.8393452585512442
A 5 1 0.3772631653874754
A 9 1 0.21203349751764478
B 2 1 1.0491224187779338
B 5 1 0.09386572432115159
B 9 1 0.6959331175663124


In [168]:
# average using peakareatop
# impose only one concentration bound at a time to check for infeasible bounds
all_data={}
Uf = 0.0 #Uncertainity factor 
DEFAULT_LB = 1e-6
DEFAULT_UB = 2e-2 #20 mM, which is 2x higher than the 10 mM default bound suggested in the Noor et al. 2014 paper.

ref_conc={}
for i, dat in Met_data.iterrows():
    if dat['Timepoint']==5 and dat['Sample']=='B':
        if dat['KEGG_ID'] not in ref_conc:
            ref_conc[dat['KEGG_ID']]={}
        if dat['Replicate']==1:
            ref_conc[dat['KEGG_ID']]=dat['amount']
        else:
            ref_conc[dat['KEGG_ID']]=(ref_conc[dat['KEGG_ID']]+dat['amount'])/2
for sm in [0,1,2,6,7,8]:#range(12): # divide the samples into datasets
    all_data[sm]={}
    if sm < 6:
        smple = 'A'
    else:
        smple = 'B'
    if (sm)%3==0:
        tym = 2
    elif (sm)%3==1:
        tym = 5
    else:
        tym = 9
    if sm<3 or (sm<9 and sm>5):
        rep = 1
    else:
        rep = 2
    #print(smple,tym,rep)
    for i, cpd in Met_bound.iterrows():  # reset bounds at the start of each iteration
        pp.bounds.SetBounds(cpd['KEGG_ID'],cpd['Concentration:Min'],cpd['Concentration:Max'] )
    for i2, dat2 in Met_data_WT.iterrows():
        if dat2['KEGG_ID'] not in ['C00044','C00035','C00002','C00004','C00005']:
            #pp.bounds.SetBounds(dat['KEGG_ID'],dat['Concentration']*.50,dat['Concentration']*1.5)
            lb={}
            ub={}
            for i, dat in Met_data.iterrows():
                try:
                    if (dat2['KEGG_ID']==dat['KEGG_ID'])and(dat['Sample']==smple and dat['Timepoint']==tym) and (dat['KEGG_ID'] not in ['C00035',"C00044","C00002","C00004","C00005"]):
                        all_data[sm][i]={}
                        if dat['KEGG_ID'] not in lb:
                            lb[dat['KEGG_ID']]={}
                        if dat['KEGG_ID'] not in ub:
                            ub[dat['KEGG_ID']]={}
                        if not lb[dat['KEGG_ID']]:
                            lb[dat['KEGG_ID']] = dat['amount']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1-Uf)
                        else:
                            lb[dat['KEGG_ID']] = (lb[dat['KEGG_ID']]+dat['amount']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1-Uf))/2
                            #if lb[dat['KEGG_ID']] < DEFAULT_LB:    
                                #lb[dat['KEGG_ID']] = DEFAULT_LB

                            #if lb[dat['KEGG_ID']] > DEFAULT_UB:
                                #lb[dat['KEGG_ID']] = DEFAULT_UB
                        if not ub[dat['KEGG_ID']]: 
                            ub[dat['KEGG_ID']] = dat['amount']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1+Uf)
                        else:
                            ub[dat['KEGG_ID']] = (ub[dat['KEGG_ID']]+dat['amount']*dat2['Concentration']/(ref_conc[dat['KEGG_ID']]) *(1+Uf))/2
                            #if ub[dat['KEGG_ID']] < DEFAULT_LB:
                                #ub[dat['KEGG_ID']] = DEFAULT_LB

                            #if ub[dat['KEGG_ID']] > DEFAULT_UB:
                                #ub[dat['KEGG_ID']] = DEFAULT_UB
                        #if dat['KEGG_ID']=='C00085':
                        #print(dat['KEGG_ID'],ref_conc[dat['KEGG_ID']],dat2['Concentration'],lb[dat['KEGG_ID']],ub[dat['KEGG_ID']],dat['amount_int'])
                        pp.bounds.SetBounds(dat['KEGG_ID'], lb[dat['KEGG_ID']], ub[dat['KEGG_ID']])
                        #print(dat['KEGG_ID'], lb[dat['KEGG_ID']], ub[dat['KEGG_ID']])
                except:
                    print("ref not found for",dat['KEGG_ID'])
                    #print(dat['KEGG_ID'], lb, ub)
                if tym  == 2 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 5.12/46, 5.12/46)
                    pp.bounds.SetBounds('C00033', 2.87/1000, 2.87/1000)
                elif tym ==5 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 20.21/46, 20.21/46)
                    pp.bounds.SetBounds('C00033', 4.25/1000, 4.25/1000)
                elif tym ==9 and smple == 'A':
                    pp.bounds.SetBounds('C00469', 40.38/46, 40.38/46)
                    pp.bounds.SetBounds('C00033', 4.39/1000, 4.39/1000)
                if tym  == 2 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.2/46, 0.2/46)
                    pp.bounds.SetBounds('C00033', 3.21/1000, 3.21/1000)
                elif tym ==5 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.47/46, 0.47/46)
                    pp.bounds.SetBounds('C00033', 8.61/1000, 8.61/1000)
                elif tym ==9 and smple == 'B':
                    pp.bounds.SetBounds('C00469', 0.58/46, 0.58/46)
                    pp.bounds.SetBounds('C00033', 9.21/1000, 9.21/1000)
    mdf_data = pp.calc_mdf()
    #print(smple,tym,rep,dat['KEGG_ID'],mdf_data.mdf)
    atp_ratio=1
    gtp_ratio=1
    nadh_ratio=1
    for c in mdf_data.compound_data:
        if c.compound in ['C00002']:
            atp_ratio=atp_ratio*c.concentration
        elif c.compound in ['C00008']:
            atp_ratio=atp_ratio/c.concentration
        elif c.compound in ['C00003']:
            nadh_ratio=nadh_ratio/c.concentration
        elif c.compound in ['C00004']:
            nadh_ratio=nadh_ratio*c.concentration
        elif c.compound in ['C00035']:
            gtp_ratio=gtp_ratio/c.concentration
        elif c.compound in ['C00044']:
            gtp_ratio=gtp_ratio*c.concentration
        #print(c.compound, c.concentration,(mdf_data.model.concentration_bounds.GetBoundTuple(c.compound)),c.shadow_price)
    #print('atp_ratio =',atp_ratio)
    #print('gtp_ratio =',gtp_ratio)
    #print('nadh_ratio =',nadh_ratio)
    print(smple,tym,rep,mdf_data.mdf)
    #print("__________________")
    all_data[sm]['sample']=smple
    all_data[sm]['timepoint']=tym
    all_data[sm]['replicate']=rep
    #all_data[sm][cpd['KEGG_ID']]['metab']=dat['KEGG_ID']
    all_data[sm]['mdf']=mdf_data
    #except:
    #print("no ref for", dat2['KEGG_ID'])

A 2 1 0.8390975578212689
A 5 1 0.3772631653874825
A 9 1 0.21203349751763767
B 2 1 1.0491224187779409
B 5 1 0.09386572432115159
B 9 1 0.6959331175663301
